In [ ]:


from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import Counter
from tqdm import tqdm

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/ML Final Project/cohort/los/admission_only_true/LOS_WEEKS_adm_train_med7.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ML Final Project/cohort/los/admission_only_true/LOS_WEEKS_adm_test_med7.csv')
val_df = pd.read_csv('/content/drive/MyDrive/ML Final Project/cohort/los/admission_only_true/LOS_WEEKS_adm_val_med7.csv')

In [ ]:
train_df.head()

,id,subject_id,gender,dob,admittime,ethnicity,text,los_label,all_entities
0,155297,41976,M,2136-07-28 00:00:00,2201-11-16 23:00:00,HISPANIC/LATINO - PUERTO RICAN,CHIEF COMPLAINT: Decreased responsiveness Maj...,0,"[('coumadin', 'DRUG'), ('Cefepime', 'DRUG'), (..."
1,168150,26316,F,2114-01-14 00:00:00,2185-12-16 15:46:00,UNKNOWN/NOT SPECIFIED,"CHIEF COMPLAINT: Fatigue, wide complex tachyca...",1,"[('propafenone', 'DRUG'), ('Digoxin', 'DRUG'),..."
2,154015,57593,M,2085-04-04 00:00:00,2144-01-13 18:55:00,WHITE,CHIEF COMPLAINT: preop CABG\n\nPRESENT ILLNESS...,2,"[('coumadin', 'DRUG'), ('Albuterol', 'DRUG'), ..."
3,145268,1217,M,2072-02-15 00:00:00,2125-04-05 16:55:00,BLACK/AFRICAN AMERICAN,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This is ...,1,"[('Hydrochlorothiazide', 'DRUG'), ('25 mg', 'S..."
4,170521,17144,M,2123-10-03 00:00:00,2195-07-23 10:27:00,UNKNOWN/NOT SPECIFIED,CHIEF COMPLAINT: worsening shortness of breath...,2,"[('atenolol', 'DRUG'), ('25', 'STRENGTH'), ('3..."


In [ ]:
train_df['all_entities'].head()

,all_entities
0,"[('coumadin', 'DRUG'), ('Cefepime', 'DRUG'), (..."
1,"[('propafenone', 'DRUG'), ('Digoxin', 'DRUG'),..."
2,"[('coumadin', 'DRUG'), ('Albuterol', 'DRUG'), ..."
3,"[('Hydrochlorothiazide', 'DRUG'), ('25 mg', 'S..."
4,"[('atenolol', 'DRUG'), ('25', 'STRENGTH'), ('3..."


In [ ]:
tqdm.pandas()

def extract_drugs(all_entities):
    return [entity[0] for entity in all_entities if entity[1] == 'DRUG']

train_df['all_entities'] = train_df['all_entities'].apply(eval)
train_df['drugs_list'] = train_df['all_entities'].apply(extract_drugs)

In [ ]:
all_drugs = [drug for drug_list in train_df['drugs_list'] for drug in drug_list]

# Construct the frequency map
drug_frequency = Counter(all_drugs)

# Convert to a dictionary or display as needed
# print(drug_frequency)
frequency_dict = dict(drug_frequency)
sorted_dict = dict(sorted(frequency_dict.items(), key=lambda item: item[1], reverse=True))


In [ ]:
len(frequency_dict)

13359

In [ ]:
test_df['all_entities'] = test_df['all_entities'].apply(eval)
test_df['drugs_list'] = test_df['all_entities'].apply(extract_drugs)

val_df['all_entities'] = val_df['all_entities'].apply(eval)
val_df['drugs_list'] = val_df['all_entities'].apply(extract_drugs)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader


tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


model.eval()
model = model.to('cuda')


def generate_embeddings(texts, batch_size=32):
    embeddings = []
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)

    for batch in tqdm(dataloader, desc="Generating embeddings"):

        tokens = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to('cuda')
        with torch.no_grad():
            output = model(**tokens)

        batch_embeddings = output.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.extend(batch_embeddings)

    return embeddings


drug_names = list(frequency_dict.keys())


batch_size = 64
embeddings = generate_embeddings(drug_names, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating embeddings:   3%|▎         | 6/209 [00:01<00:37,  5.37it/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating embeddings: 100%|██████████| 209/209 [00:11<00:00, 18.52it/s]


In [ ]:
!pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import faiss
from tqdm import tqdm
import pickle

In [ ]:

embeddings_array = np.array(embeddings).astype('float32')
print(len(embeddings_array))


cluster_variations = [8, 16, 32, 64, 128, 256, 512]


embedding_dim = embeddings_array.shape[1]

res = faiss.StandardGpuResources()
faiss_index = faiss.IndexFlatL2(embedding_dim)
gpu_index = faiss.index_cpu_to_gpu(res, 0, faiss_index)


for num_clusters in cluster_variations:
  kmeans = faiss.Clustering(embedding_dim, num_clusters)
  kmeans.niter = 20
  kmeans.max_points_per_centroid = len(frequency_dict)
  kmeans.train(embeddings_array, gpu_index)


  centroids = faiss.vector_to_array(kmeans.centroids).reshape(num_clusters, embedding_dim)
  _, cluster_ids = gpu_index.search(embeddings_array, 1)
  cluster_ids = cluster_ids.flatten()
  word_to_cluster = {drug_names[i]: int(cluster_ids[i]) for i in range(len(drug_names))}
  print(word_to_cluster)

  train_df[f'{num_clusters}_cluster_ids'] = train_df['drugs_list'].apply(lambda drugs: [word_to_cluster.get(drug) for drug in drugs]
  def assign_clusters(diseases, word_to_cluster):
      return [word_to_cluster.get(disease, -1) for disease in diseases if disease in word_to_cluster]

  test_df[f'{num_clusters}_cluster_ids'] = test_df['drugs_list'].apply(lambda drugs: assign_clusters(drugs, word_to_cluster))
  val_df[f'{num_clusters}_cluster_ids'] = val_df['drugs_list'].apply(lambda drugs: assign_clusters(drugs, word_to_cluster))


13359
{'coumadin': 0, 'Cefepime': 6, 'ceftriaxone': 6, 'meropenem': 1, 'cipro': 6, 'zosyn': 6, 'cefepime': 6, 'vanco': 6, 'NS': 4, 'albuterol sulfate': 3, 'ipratropium bromide': 3, 'baclofen': 6, 'duloxetine': 3, 'fentanyl': 6, 'gabapentin': 1, 'levothyroxine': 3, 'mirtazapine': 1, 'acetaminophen': 3, 'ascorbic acid': 7, 'miconazole nitrate': 1, 'senna': 7, 'lansoprazole': 1, 'bisacodyl': 3, 'ferrous sulfate': 3, 'alum-mag hydroxide-simeth': 6, 'docusate sodium': 0, 'enoxaparin': 6, 'Insulin': 7, 'warfarin': 6, 'acetylcysteine': 3, 'Milk of Magnesia': 3, 'Glucerna': 7, 'multivitamin': 6, 'Novolin': 0, 'propafenone': 1, 'Digoxin': 6, 'amp bicarb': 0, 'D50': 7, 'insulin': 7, 'kayexalate': 1, 'prednisone': 1, 'Levoflox': 0, 'Fluoxetine': 1, 'dulcolax': 1, 'digoxin': 6, 'risperdal': 2, 'lipitor': 7, 'protonix': 6, 'duonebs': 2, 'xanax': 6, 'lasix': 0, 'advair': 7, 'glipizide': 1, 'Albuterol': 1, 'Amlodipine': 1, 'Lasix': 0, 'Glipizide': 1, 'Lisinopril': 1, 'Metformin': 1, 'Nitroglycerin': 

In [ ]:
train_df['8_cluster_ids'].head()

,8_cluster_ids
0,"[0, 6, 6, 1, 6, 1, 6, 6, 6, 4, 0, 3, 3, 6, 3, ..."
1,"[1, 6, 1, 6, 1, 0, 7, 7, 1, 1, 0, 0, 1, 1, 6, ..."
2,"[0, 1, 1, 6, 0, 1, 1, 1, 3, 3, 5, 6, 0, 3]"
3,"[3, 7, 6]"
4,"[1, 5]"


In [ ]:
train_df.columns

Index(['id', 'subject_id', 'gender', 'dob', 'admittime', 'ethnicity', 'text',
       'los_label', 'all_entities', 'drugs_list', '8_cluster_ids',
       '16_cluster_ids', '32_cluster_ids', '64_cluster_ids', '128_cluster_ids',
       '256_cluster_ids', '512_cluster_ids'],
      dtype='object')

In [ ]:


def create_cluster_one_hot(cluster_ids, num_clusters):
    one_hot_vector = [0] * num_clusters
    for cluster_id in set(cluster_ids):
        if cluster_id != -1 and cluster_id < num_clusters:
            one_hot_vector[cluster_id] = 1
    return one_hot_vector

for num_clusters in cluster_variations:
    train_df[f'{num_clusters}_cluster_onehot'] = train_df[f'{num_clusters}_cluster_ids'].apply(lambda x: create_cluster_one_hot(x, num_clusters))
    test_df[f'{num_clusters}_cluster_onehot'] = test_df[f'{num_clusters}_cluster_ids'].apply(lambda x: create_cluster_one_hot(x, num_clusters))
    val_df[f'{num_clusters}_cluster_onehot'] = val_df[f'{num_clusters}_cluster_ids'].apply(lambda x: create_cluster_one_hot(x, num_clusters))

In [ ]:
train_df['64_cluster_onehot']

,64_cluster_onehot
0,"[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ..."
1,"[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, ..."
2,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
...,...
30416,"[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30417,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ..."
30418,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30419,"[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [ ]:


for split, df in zip(['train', 'test', 'val'], [train_df, test_df, val_df]):
      # Save to CSV
    df.to_csv(f'/content/drive/MyDrive/ML Final Project/cohort/los/admission_only_true/{split}_med7_knn_onehot.csv', index=False)

In [ ]:

cluster_variations = [8, 16, 32, 64, 128, 256, 512]

# Prepare the data
X_train = {}
X_test = {}
X_val = {}

for num_clusters in cluster_variations:
    X_train[f'{num_clusters}'] = np.array(train_df[f'{num_clusters}_cluster_onehot'].tolist())
    X_test[f'{num_clusters}'] = np.array(test_df[f'{num_clusters}_cluster_onehot'].tolist())
    X_val[f'{num_clusters}'] = np.array(val_df[f'{num_clusters}_cluster_onehot'].tolist())

y_train = train_df['los_label'].values
y_test = test_df['los_label'].values
y_val = val_df['los_label'].values

In [ ]:
print(X_train['8'].shape)
print(X_test['8'].shape)
print(X_val['8'].shape)

(30421, 8)
(8797, 8)
(4391, 8)


In [ ]:

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report


models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    # 'SVM Classifier': SVC(),
    'XGBoost': XGBClassifier(),
    'Gradient Boosted Trees': GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.01,
        max_depth=200,
        random_state=42
    )
}

def store_results(num_clusters, results, y_test, y_pred):
    # Calculate metrics
    precision_per_class = precision_score(y_test, y_pred, average=None)
    recall_per_class = recall_score(y_test, y_pred, average=None)
    accuracy = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')

    # Store results
    results.append([
        num_clusters,
        precision_per_class[0], precision_per_class[1], # Assuming binary classification
        recall_per_class[0], recall_per_class[1], # Assuming binary classification
        accuracy,
        f1_macro,
        f1_weighted
    ])
cluster_variations = [64, 256, 512]
for num_clusters in cluster_variations:
    print(f"Results for {num_clusters} clusters:")

    X_train = np.array(train_df[f'{num_clusters}_cluster_onehot'].tolist())
    X_test = np.array(test_df[f'{num_clusters}_cluster_onehot'].tolist())
    X_val = np.array(val_df[f'{num_clusters}_cluster_onehot'].tolist())

    results = []
    for model_name, model in models.items():
        print(f"  {model_name}:")

        # Experiment 1: Without SMOTE
        print("    Without SMOTE:")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(classification_report(y_test, y_pred))

        # Calculate metrics
        precision_per_class = precision_score(y_test, y_pred, average=None)
        recall_per_class = recall_score(y_test, y_pred, average=None)
        accuracy = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average='macro')
        f1_weighted = f1_score(y_test, y_pred, average='weighted')

        # Store results
        results.append([
            num_clusters,
            0,
            precision_per_class[0], precision_per_class[1], # Assuming binary classification
            recall_per_class[0], recall_per_class[1], # Assuming binary classification
            accuracy,
            f1_macro,
            f1_weighted
        ])


        # Experiment 2: With SMOTE
        print("    With SMOTE:")
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
        model.fit(X_train_resampled, y_train_resampled)
        y_pred_smote = model.predict(X_test)
        print(classification_report(y_test, y_pred_smote))

         # Calculate metrics
        precision_per_class = precision_score(y_test, y_pred, average=None)
        recall_per_class = recall_score(y_test, y_pred, average=None)
        accuracy = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average='macro')
        f1_weighted = f1_score(y_test, y_pred, average='weighted')

        # Store results
        results.append([
            num_clusters,
            1,
            precision_per_class[0], precision_per_class[1], # Assuming binary classification
            recall_per_class[0], recall_per_class[1], # Assuming binary classification
            accuracy,
            f1_macro,
            f1_weighted
        ])


    # Create a DataFrame from the results
    results_df = pd.DataFrame(results, columns=[
      'num_clusters',
      'SMOTE',
      'precision_class_0', 'precision_class_1',
      'recall_class_0', 'recall_class_1',
      'accuracy', 'f1_macro', 'f1_weighted'
    ])

    # Save the results to a CSV file
    results_df.to_csv('/content/drive/MyDrive/ML Final Project/cohort/mp/admission_only_true/classification_results.csv', index=False)


Results for 64 clusters:
  Logistic Regression:
    Without SMOTE:
              precision    recall  f1-score   support

           0       0.24      0.01      0.01      1121
           1       0.39      0.85      0.53      3328
           2       0.33      0.16      0.21      2692
           3       0.29      0.05      0.08      1656

    accuracy                           0.38      8797
   macro avg       0.31      0.26      0.21      8797
weighted avg       0.34      0.38      0.28      8797

    With SMOTE:
              precision    recall  f1-score   support

           0       0.16      0.44      0.24      1121
           1       0.42      0.29      0.34      3328
           2       0.34      0.17      0.23      2692
           3       0.25      0.32      0.28      1656

    accuracy                           0.28      8797
   macro avg       0.29      0.31      0.27      8797
weighted avg       0.33      0.28      0.28      8797

  XGBoost:
    Without SMOTE:
              pre

KeyboardInterrupt: 

In [ ]:

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample


def balance_classes(df, label_column):
    """
    Oversamples each class in the label column to make all classes equally represented.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing data and the label column.
        label_column (str): The name of the column containing class labels.

    Returns:
        pd.DataFrame: A balanced DataFrame with equal representation of all classes.
    """
    # Determine the maximum class count
    max_count = df[label_column].value_counts().max()

    # List to hold oversampled data
    oversampled_dfs = []

    # Oversample each class
    for label in df[label_column].unique():
        class_subset = df[df[label_column] == label]
        oversampled_subset = resample(
            class_subset,
            replace=True,  # Sampling with replacement
            n_samples=max_count,  # Oversample to match the max class count
            random_state=42  # For reproducibility
        )
        oversampled_dfs.append(oversampled_subset)

    # Combine all oversampled subsets
    balanced_df = pd.concat(oversampled_dfs, ignore_index=True)

    # Shuffle the balanced DataFrame
    balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

    return balanced_df





models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(),
    'Gradient Boosted Trees': GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.01,
        max_depth=20,
        random_state=42
    )
}


X_test = np.array(test_df[f'{num_clusters}_cluster_onehot'].tolist())
y_test = test_df['los_label'].values

balanced_train_df = balance_classes(train_df, 'los_label')
X_train_balanced = np.stack(balanced_train_df[f'{num_clusters}_cluster_onehot'].values)
y_train_balanced = balanced_train_df['los_label'].values


num_clusters = 256
results = []
for model_name, model in models.items():
    print(f"  {model_name}:")

    print("    With Resampling:")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

    # Calculate metrics
    precision_per_class = precision_score(y_test, y_pred, average=None)
    recall_per_class = recall_score(y_test, y_pred, average=None)
    accuracy = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')

    # Store results
    results.append([
        model_name,
        precision_per_class[0], precision_per_class[1], # Assuming binary classification
        recall_per_class[0], recall_per_class[1], # Assuming binary classification
        accuracy,
        f1_macro,
        f1_weighted
    ])


# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=[
      'Model',
      'precision_class_0', 'precision_class_1',
      'recall_class_0', 'recall_class_1',
      'accuracy', 'f1_macro', 'f1_weighted'
    ])

# Save the results to a CSV file
results_df.to_csv('/content/drive/MyDrive/cohort/mp/admission_only_true/classification_results.csv', index=False)

  Logistic Regression:
    With Resampling:
              precision    recall  f1-score   support

           0       0.19      0.40      0.26      1121
           1       0.40      0.21      0.28      3328
           2       0.32      0.31      0.31      2692
           3       0.26      0.34      0.29      1656

    accuracy                           0.29      8797
   macro avg       0.29      0.31      0.29      8797
weighted avg       0.32      0.29      0.29      8797

  Random Forest:
    With Resampling:
              precision    recall  f1-score   support

           0       0.24      0.20      0.22      1121
           1       0.40      0.38      0.39      3328
           2       0.32      0.30      0.31      2692
           3       0.23      0.31      0.27      1656

    accuracy                           0.32      8797
   macro avg       0.30      0.30      0.30      8797
weighted avg       0.32      0.32      0.32      8797

  XGBoost:
    With Resampling:
              pr

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/cohort/mp/admission_only_true'